In [1]:
from langchain_core.documents import Document
from langchain_core.tools import tool
from typing import List, Any
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 웹 검색 리트리버 클래스 정의
class WebSearchRetriever(BaseRetriever):
    search_wrapper: DuckDuckGoSearchAPIWrapper

    def __init__(self, search_wrapper: DuckDuckGoSearchAPIWrapper, **kwargs: Any):
        super().__init__(search_wrapper=search_wrapper, **kwargs)

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        results = self.search_wrapper.results(query, max_results=10)
        if not results:
            return [Document(page_content="관련 정보를 찾을 수 없습니다.")]
        
        formatted_docs = []
        for result in results:
            doc = Document(
                page_content=f'<Document href="{result.get("link", "")}"/>\n{result.get("snippet", "")}\n</Document>',
                metadata={
                    "source": "web search", 
                    "url": result.get("link", ""), 
                    "title": result.get("title", "")
                }
            )
            formatted_docs.append(doc)
        return formatted_docs

@tool
def web_search(query: str, search_period: str = 'm') -> List[Document]:
    """
    웹 검색을 수행하고 결과를 Document 리스트 형태로 반환하는 함수.
    search_period: 검색 기간 (d: 1일, w: 1주, m: 1달, y: 1년)
    """
    ddg_search_wrapper = DuckDuckGoSearchAPIWrapper(time=search_period)
    web_retriever = WebSearchRetriever(search_wrapper=ddg_search_wrapper)

    docs = web_retriever.invoke(query)

    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 정보를 찾을 수 없습니다.")]

In [2]:
# 도구 목록을 정의 
tools = [web_search]

In [3]:
query = "KT 소액 결제 해킹 사건의 원인과 대책?"

tools[0].invoke(query)

[Document(metadata={'source': 'web search', 'url': 'https://www.chosun.com/economy/tech_it/2025/09/20/BY4FMCIIY5GLRN5ZQHYQYVGGXA/', 'title': '소액결제 사태 KT , 서버까지 뚫려… 3일 뒤에야 신고 | 조선일보'}, page_content='<Document href="https://www.chosun.com/economy/tech_it/2025/09/20/BY4FMCIIY5GLRN5ZQHYQYVGGXA/"/>\n민관합동조사단의 조사는 KT 가입자들 사이에서 발생한 무단 소액 결제 사건과 이번 해킹 사건의 연관성을 규명하는 데 집중될 것으로 보인다.\n</Document>'),
 Document(metadata={'source': 'web search', 'url': 'https://v.daum.net/v/20250917172639524', 'title': "KT 소액결제 해킹 용의자 2명 검거… '사건 전말' 드러나나"}, page_content='<Document href="https://v.daum.net/v/20250917172639524"/>\nKT 무단 소액 결제 사건의 용의자로 지목된 40대 중국 동포 두 명이 경찰에 붙잡혔다.\n</Document>'),
 Document(metadata={'source': 'web search', 'url': 'https://www.tiktok.com/discover/팬딩-소액결제', 'title': '팬딩 소액결제 | TikTok'}, page_content='<Document href="https://www.tiktok.com/discover/팬딩-소액결제"/>\nKT 이용자를 노린 소액 결제 해킹 사건의 범인이 검거되었습니다. 사건의 경과와 현 시점의 문제를 알아보세요. KT 결제 해킹 사건과 중국인 검거.\n</Document>'),
 Document(metadata={'source': 'we